# SNAREseq Mouse

This IPython notebook demonstrates how to preprocess ATAC-seq data to get the closest downstream genes within  1k threshold distance of the chromatin regions.

<span class="mark">1)</span> Convert the genes' ensemble ids to gene names in the promoter file, as we have gene symbols in the scRNAseq data of SNAREseq to compare later. 

In [21]:

gene_list = []
with open('/home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters.bed') as file:
    for line in file:
        gene_name = line.split('\t')[-3]
        gene_list.append(gene_name)

In [22]:
with open('/home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_genes_ens.bed', 'w') as file:
    for gene in gene_list:
        file.write(gene + '\n')

Then, I used the below website to convert ensemble IDs to gene symbols:https://www.biotools.fr/mouse/ensembl_symbol_converter

In [23]:
! head /home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_genes_ens_to_gene_symbol.bed

ENSMUSG00000102693	4933401J01Rik
ENSMUSG00000064842	Gm26206
ENSMUSG00000051951	Xkr4
ENSMUSG00000102851	Gm18956
ENSMUSG00000103377	Gm37180
ENSMUSG00000104017	Gm37363
ENSMUSG00000103025	Gm37686
ENSMUSG00000089699	Gm1992
ENSMUSG00000103201	Gm37329
ENSMUSG00000103147	Gm7341


Then, I need to treat the above file as a dictionary to convert ensemble IDs in the promoter file:

In [24]:
gene_dict = dict()
with open('/home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_genes_ens_to_gene_symbol.bed', 'r') as file:
    for line in file:
        split_line = line.split('\t')
        gene_dict[split_line[0]] = split_line[-1][:-1] 

In [28]:
with open('/home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters.bed') as file:
    with open('/home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_chr_fixed_symbols.bed', 'a') as out_f:
        for line in file:
            # chr1	3072251	3074253	ENSMUSG00000102693	+
            split_line = line.split('\t')
            chrom = split_line[0]
            if 'CHR' not in chrom:
                chrom = 'chr' + chrom
            new_line = chrom + '\t' + split_line[1] + '\t' + split_line[2] + '\t' + gene_dict[split_line[3]] + '\t' + split_line[-1]
            out_f.write(new_line)

<span class="mark">2)</span> Sort the **promoter** file and also **peak** file

In [44]:
# Sort the promoter file
! sort -k 1,1 -k2,2n /home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_chr_fixed_symbols.bed  > /home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_chr_fixed_symbols_sorted.bed

In [37]:
# Make TAB-delimited peak file 5k
with open('/home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/GSE126074_P0_BrainCortex_SNAREseq_chromatin.peaks.tsv', 'r') as f:
    with open('/home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/peaks.bed', 'a') as f_out:
        for line in f:
            # chr:start-end
            split_line = line.replace(':','-').split('-')
            f_out.write('\t'.join(split_line))
        

In [45]:
# sort the peak file 5k
! sort -k 1,1 -k2,2n /home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/peaks.bed > /home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/peaks_sorted.bed

<span class="mark">Bedtools closest</span>

    bedtools closest -a /home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/peaks_sorted.bed -b /home/ejafari/alignment/downstream/data/atac/prom/mouse_promoters_chr_fixed_symbols_sorted.bed -iu -D b -t first > /home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/find_closest_gene_downstream_strand_first.bed

Removed about 600 lines from promoter file that had strange CHR names in them, ow bedtools closest won't proceed and returns an error instead.

## ATAC

### Create ATAC count matrix

In [56]:
from scipy import io
import pandas as pd
input_dir = "/home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/"


matrix = io.mmread(input_dir + 'GSE126074_P0_BrainCortex_SNAREseq_chromatin.counts.mtx')

# Read regions
peak_list = []
with open(input_dir + "GSE126074_P0_BrainCortex_SNAREseq_chromatin.peaks.tsv", "r") as filestream:
    for line in filestream:
        peak_list.append(line[:-1])
#         currentline = line.split("\t")
#         peak_list.append(currentline[0] + ":" + currentline[1] + "-" + currentline[2][:-1])


# Read cells
cell_list = []
with open(input_dir + "GSE126074_P0_BrainCortex_SNAREseq_chromatin.barcodes.tsv", "r") as filestream:
    for line in filestream:
        currentline = line.split(",")
        cell_list.append(currentline[0][:-1])
        
df = pd.DataFrame(matrix.toarray(), index=peak_list, columns=cell_list)
df.to_csv(input_dir + 'scATACseq.csv', index=True, header=True)



In [57]:
df.head()

,TGGAATTTTCTC,CCAACAAACGCG,TGCGCATAGCCG,CTGTTTCCCACC,ACAGTCTACATG,CGCACTTGCGAG,GCAACCTGAACA,TAAAACCCACCA,AAGTTGACCAAG,CCGTGAGCTGCA,...,GAAGCGAGGTCT,GCTTATTTATAC,GTTGACCCCCTC,CGAATTAATCAG,ACCGCTTTTTAA,TTTAGGCCATGT,GGGTCTTTTCGA,TTCACGTTGACC,CACCTCCAGCGA,CTGGGCATCATT
chr1:3012650-3012823,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chr1:3012853-3013002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chr1:3030589-3030826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chr1:3071552-3071701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chr1:3078770-3078919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



Run <span class="mark"> cisTopic </span><i class="fa fa-lightbulb-o "></i> on the ATAC-seq data.

In [4]:
# Move this ATACseq.csv file to the Hulk, and run cisTopic on it.
# R code for Topic Modeling
! head -40 cisTopic.R

library(Matrix)
suppressWarnings(library(cisTopic, lib.loc ="~/.local/bin"))

atac_data <- read.csv(file = 'ATACseq.csv', row.names = 1, header= TRUE, sep = ',')


atac_mat <- data.matrix(atac_data)
RowName <- rownames(atac_data)
ColName <- colnames(atac_data)
Indrow = row(atac_data)[which(!atac_data == 0)]
Indcol = col(atac_data)[which(!atac_data == 0)]
Val = atac_mat[ which(!atac_mat == 0)]
atac_SpaMat <-sparseMatrix(
  i = Indrow, 
  j = Indcol, 
  x = Val,
  dims = c(nrow(atac_data), ncol(atac_data)), 
  dimnames = list(RowName, ColName)
)

cisTopicObject <- createcisTopicObject(atac_SpaMat, project.name='Mus')
rm(atac_SpaMat)
rm(atac_mat)

cisTopicObject <- runWarpLDAModels(cisTopicObject, topic=c(2:15, 20, 25, 35, 40, 45, 50), seed=123, nCores=17, addModels=FALSE)

cisTopicObject <- selectModel(cisTopicObject, type='derivative')

#cisTopicObject <- runUmap(cisTopicObject, target='cell')
#plotFeatures(cisTopicObject, method='Umap', target='cell', topic_contr='Probability', dim=2)


## 1k threshold distance

In [6]:
# stringent threshold for distance when finding the closest gene (1k)
__author__ = "Elham Jafari"
__email__ = "ejafari@indiana.edu"

import re 
import pickle
import pandas as pd
import matplotlib.pyplot as plt


def convert_to_closest_gene(pred_f, input_dir, closest_file, out_f, threshold=1000):
    pred_matrix_atac = pd.read_csv(pred_f, delimiter='\t')
    print(pred_matrix_atac.sum(axis=0))
    print(pred_matrix_atac.sum(axis=1))
    print('----------------------------------------------\n')
    counter = 0
    closest_gene_dict = dict()
    # Keeps the genome distance for each line of the file
    dist_list = []
    regions_above_1k = []
    with open(input_dir + closest_file, 'r') as f:
        for line in f:
            fields = line.split('\t')
            # Distance is the last field in each line
            distance = int(fields[-1].strip('\n'))
            region = fields[0] + ":" + str(fields[1]) + "-" + str(fields[2]) 
            # Threshold of 1k for genome distance
            if abs(distance) <= threshold:
                dist_list.append(distance)
                gene = fields[-3]
                closest_gene_dict[region] = gene
                counter = counter + 1
            # Save the regions that are filtered out
            elif region in pred_matrix_atac.index:
                regions_above_1k.append(region)


    print('-------------------------------------------------------')
    # Change the region to the closest gene name
    pred_matrix_atac.rename(closest_gene_dict, inplace=True)
    print("Shape of the whole region DataFrame: ", pred_matrix_atac.shape)
    print("Number of regions to be filtered out (Greater than 1k distance): ", len(regions_above_1k))

    # Remove rows that have regions with distance more than 1k
    pred_matrix_atac = pred_matrix_atac.drop(regions_above_1k)
    print("Shape of the region DataFrame after filtering out regions greater than 1k: ", pred_matrix_atac.shape)

    # Sum up similar closest genes
    pred_matrix_atac.index.name = 'closest_gene'
    pred_matrix_atac = pred_matrix_atac.groupby(by='closest_gene').sum()
    
    # Remove . which means when no ties are found for a region
    pred_matrix_atac.drop(['.'], inplace=True)

    print("Shape of the region DataFrame after filtering out and summing up similar genes: ", pred_matrix_atac.shape)
    pred_matrix_atac.to_csv(input_dir + out_f, index=True, header=True, sep=',')
    return pred_matrix_atac

In [ ]:
# Run this using http://localhost:8000/edit/alignment/downstream/data/SNAREseq/Mouse/5k/closest_gene.py

pred_f = '/home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/pred_atac.csv'
input_dir = "/home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/"
closest_file = "find_closest_gene_downstream_strand_first.bed"
out_f = 'pred_matrix_closest_genes_1k_downstream_strand_first_prom.csv'

# "convert_to_closest_gene" has been changed slightly in comparisonto pipeline_preprocesing.ipynb
atac = convert_to_closest_gene(pred_f, input_dir, closest_file, out_f, threshold=1000)

Shape of the whole region DataFrame:  (229429, 5081)

Number of regions to be filtered out (Greater than 1k distance):  204081

Shape of the region DataFrame after filtering out regions greater than 1k:  (25348, 5081)

Shape of the region DataFrame after filtering out and summing up similar genes:  (19598, 5081)

## RNA

In [1]:
input_dir = "/home/ejafari/alignment/downstream/data/SNAREseq/Mouse/5k/"


from scipy import io
import pandas as pd

# io.mmwrite('test_io.mtx', matrix)
scRNAseq_adrs = input_dir + 'GSE126074_P0_BrainCortex_SNAREseq_cDNA.counts.mtx'

matrix = io.mmread(scRNAseq_adrs)

# Read genes
gene_list = []
with open(input_dir + "GSE126074_P0_BrainCortex_SNAREseq_cDNA.genes.tsv", "r") as filestream:
    for line in filestream:
        currentline = line.split(",")
        gene_list.append(currentline[0][:-1])

# Read cells
cell_list = []
with open(input_dir + "GSE126074_P0_BrainCortex_SNAREseq_cDNA.barcodes.tsv", "r") as filestream:
    for line in filestream:
        currentline = line.split(",")
        cell_list.append(currentline[0][:-1])
        
df = pd.DataFrame(matrix.toarray(), index=gene_list, columns=cell_list)
df.to_csv(input_dir + 'scRNAseq.csv', index=True, header=True)

In [4]:
df.head()

,GGCAATGGCCCT,GGAGTCCATGGT,GAATCCGTCCCA,GGGAAAGTATTG,GTCATCCTGAGA,GCAGAGAGCCTC,GAGTGTACCTCA,CTACACCACACC,TCAATAGGACCC,ACCTGGCCTGCC,...,TGATAACTAACA,TCCCGCAGTGGT,ATCGCCCCAGGT,ACTAGTAAGAAG,CATGTTTGCAAA,GTGCACGCGTGG,CACCTCCAGCGA,TTAGACCCGCTA,GTTTCTATTGAC,TCGCGACTCGCG
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610007P14Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B22Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009E02Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009L18Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(19322, 5081)